In [1]:
import pandas as pd
from TenIndicators.indicators import *

C:\Users\gaurav gupta\AppData\Local\Temp\ipykernel_2928\3640489074.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


ModuleNotFoundError: No module named 'MetaTrader5'

In [2]:
!pip install pandas


[notice] A new release of pip available: 22.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import pandas as pd
from TenIndicators.indicators import *

ModuleNotFoundError: No module named 'MetaTrader5'

In [4]:
import MetaTrader5 as mt5
import pandas as pd

ModuleNotFoundError: No module named 'MetaTrader5'

In [6]:
!pip install MetaTrader5

     -------------------------------------- 50.2/50.2 kB 283.7 kB/s eta 0:00:00



[notice] A new release of pip available: 22.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
import MetaTrader5 as mt5
import pandas as pd

In [8]:
import pandas as pd
from TenIndicators.indicators import *

In [10]:
df = pd.read_csv('pastdata.csv')
print(df.head())

   Unnamed: 0       converted_time        time     open     high      low  \
0           0  2016-03-09 07:00:00  1457506800  1.34358  1.34361  1.34268   
1           1  2016-03-09 08:00:00  1457510400  1.34308  1.34337  1.34229   
2           2  2016-03-09 09:00:00  1457514000  1.34249  1.34346  1.34134   
3           3  2016-03-09 10:00:00  1457517600  1.34228  1.34244  1.33991   
4           4  2016-03-09 11:00:00  1457521200  1.34084  1.34084  1.33846   

     close  spread  tick_volume  real_volume  
0  1.34306       9         1840   1063345000  
1  1.34249       9         1744   1553800000  
2  1.34231       9         4418   4627827000  
3  1.34084       9         6463   6488430000  
4  1.33976       9         6165   7256560000  


In [12]:
rate_macd = get_simple_moving_average(df, 20)

       Unnamed: 0       converted_time        time     open     high      low  \
49999       49999  2024-03-20 09:00:00  1710925200  1.35794  1.35807  1.35724   

         close  spread  tick_volume  real_volume       ema       sma  
49999  1.35751      13         1446            0  1.357155  1.357525  


In [13]:
final_rates = rate_macd


trades = pd.DataFrame(columns=['state', 'order_type', 'open_time', 
                               'open_price', 'close_time', 'close_price', 
                               'exit_type', 'macd_value_at_buy', 'signal_value_at_buy','histogram','volume', "sell_sma_greater_ema"
                               ])

In [14]:
def is_bullish_crossover_under_zero(row,index):
    prev_row = final_rates.iloc[index - 1]
    hist_trend_bullish = row['histogram'] > prev_row['histogram']
    macd_crossover = row['macd'] > row['signal']
    bullish_trend = row['signal'] < 0 and row['macd'] < 0 and macd_crossover and hist_trend_bullish
    return bullish_trend


In [18]:
def is_historgram_in_bearish_trend(index):
    if index <= 0:
        # If the index is invalid, return False
        return False
    
    # Retrieve the current and previous rows from the DataFrame
    row = final_rates.iloc[index]
    prev_row = final_rates.iloc[index - 1]
    
    # Check if the current histogram value is less than the previous one
    # This indicates a potential bearish trend
    hist_trend_bearish = row['histogram'] < prev_row['histogram']
    
    # Check if MACD is below the signal line
    macd_crossover = row['macd'] < row['signal']
    
    # Check if both MACD and signal line are above 0
    # This ensures that we are not in the oversold region
    bearish_trend = row['signal'] > 0 and row['macd'] > 0 and macd_crossover and hist_trend_bearish
    
    return bearish_trend


In [19]:
rows,columns = final_rates.shape
bought = False
buy_price = 0
tp = .0060
sl = .0009
last_bought_index = 0
trading_units = 100000
last_bought_timestamp = None
exit_from_hrs = 3

In [20]:
for index, row in final_rates.iterrows():
    
       
    if index > 30 and ((index + 1) < rows): 

        bullish_trend = is_bullish_crossover_under_zero(row, index)
        is_historgram_in_bearish = is_historgram_in_bearish_trend(index)  # Fixed function call

        current_index_last_bought_delta = index - last_bought_index
        if not bought:
            if bullish_trend:
                print("Bullish trend", row['converted_time'])
                # print("Bought",index)
                regression_data = final_rates.iloc[index - 20:index]
                buy_price = final_rates.iloc[index + 1]['open']
                last_bought_index = index
                stop_loss_price = buy_price - sl
                take_profit_price = buy_price + tp
                sma_greater_ema = row['sma'] > row['ema']
                trades.loc[len(trades)] = ['open', 'buy', final_rates.iloc[index + 1]['converted_time'], buy_price, None, None, None, row['macd'], row['signal'], row['histogram'], row['tick_volume'], None]
                last_bought_timestamp = final_rates.iloc[index + 1]['time']
                exit_time_stamp = last_bought_timestamp + (exit_from_hrs * 60 * 60)
                bought = True

        elif bought and (current_index_last_bought_delta > 3):
            if row['close'] < stop_loss_price:
                exit_type = "stop_loss"
                bought = False
                sma_greater_ema = row['sma'] > row['ema']
                trades.loc[trades['state'] == 'open', ['state', 'close_time', 'close_price', 'exit_type', "sell_sma_greater_ema"]] = ['closed', row['converted_time'], stop_loss_price, exit_type, sma_greater_ema]

            elif is_historgram_in_bearish:
                bought = False
                close_price = final_rates.iloc[index + 1]['open']
                if buy_price < close_price:
                    exit_type = "profit_on_bearish_trend"
                else:
                    exit_type = "loss_on_bearish_trend"
                sma_greater_ema = row['sma'] > row['ema']
                trades.loc[trades['state'] == 'open', ['state', 'close_time', 'close_price', 'exit_type', 'sell_sma_greater_ema']] = ['closed', final_rates.iloc[index + 1]['converted_time'], final_rates.iloc[index + 1]['open'], exit_type, sma_greater_ema]

            elif row['high'] > take_profit_price:
                bought = False
                exit_type = "take_profit"
                sma_greater_ema = row['sma'] > row['ema']
                trades.loc[trades['state'] == 'open', ['state', 'close_time', 'close_price', 'exit_type', 'sell_sma_greater_ema']] = ['closed', final_rates.iloc[index]['converted_time'], take_profit_price, exit_type, sma_greater_ema]


KeyError: 'histogram'

In [21]:
print(final_rates.columns)


Index(['Unnamed: 0', 'converted_time', 'time', 'open', 'high', 'low', 'close',
       'spread', 'tick_volume', 'real_volume', 'ema', 'sma'],
      dtype='object')
